## ALL_CNN Model

In [1]:
import tensorflow as tf
import numpy as np
import pickle
import random as rd

ImportError: Could not find 'nvcuda.dll'. TensorFlow requires that this DLL be installed in a directory that is named in your %PATH% environment variable. Typically it is installed in 'C:\Windows\System32'. If it is not present, ensure that you have a CUDA-capable GPU with the correct driver installed.

In [2]:
def unpickle(file):
   
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [4]:
x_train=np.array([])
y_train=np.array([])
for i in range(1,6):
    
    batch = unpickle("/data/cifar-10-batches-py/data_batch_%d"%(i))
    if len(x_train)==0 & len(y_train)==0:
        x_train = batch[b'data']
        y_train = batch[b'labels']
    else:
        x_train = np.concatenate((x_train, batch[b'data'])) 
        y_train = np.concatenate((y_train, batch[b'labels']))

In [5]:
y_train0 = np.empty((0,10), int)
for y in y_train:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_train0 = np.append(y_train0, np.array([indi]), axis=0)

In [6]:
test_batch = unpickle("/data/cifar-10-batches-py/test_batch")
x_test = test_batch[b'data']
y_test = test_batch[b'labels']

In [7]:
y_test0 = np.empty((0,10), int)
for y in y_test:
    indi= [1 if i==y else 0 for i in range(0,10)]
    y_test0 = np.append(y_test0, np.array([indi]), axis=0)

## Model

In [8]:
def compute_logits(x):
    """CNN-FMP network model"""
    
    #input x.shape = [,32*32*3]
    x_image = tf.reshape(x,[-1,32,32,3])
    
    #define some constants, stack size for convolutional layers
    n1=320
    n2=640
    n3=960
    n4=1280
    #n5=1600
    #n6=1920

    
    #Block1
    #cnn_1
    W_conv1 = tf.get_variable('W_conv1', shape=[2, 2, 3, n1])
    b_conv1 = tf.get_variable('b_conv1', shape=[n1])
    h_conv1 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME'), b_conv1))
    #cnn_2
    W_conv2 = tf.get_variable('W_conv2', shape=[2, 2, n1, n1])
    b_conv2 = tf.get_variable('b_conv2', shape=[n1])
    h_conv2 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME'), b_conv2))
    #cnn_3
    W_conv3 = tf.get_variable('W_conv3', shape=[2, 2, n1, n1])
    b_conv3 = tf.get_variable('b_conv3', shape=[n1])
    h_conv3 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv2, W_conv3, strides=[1, 2, 2, 1], padding='SAME'), b_conv3)
    

    
    #block_2
    #dropout rate = 0.1 
    #cnn_4
    W_conv4 = tf.get_variable('W_conv4', shape=[2, 2, n1, n2])
    b_conv4 = tf.get_variable('b_conv4', shape=[n2])
    h_conv4 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv3, W_conv4, strides=[1, 1, 1, 1], padding='SAME'), b_conv4))
    #dropout_1
    d_1=tf.nn.dropout(h_conv4,0.9)
    #cnn_5
    W_conv5 = tf.get_variable('W_conv5', shape=[2, 2, n2, n2])
    b_conv5 = tf.get_variable('b_conv5', shape=[n2])
    h_conv5 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_1, W_conv5, strides=[1, 1, 1, 1], padding='SAME'), b_conv5))
    #dropout_2
    d_2=tf.nn.dropout(h_conv5,0.9)
    #cnn_6
    W_conv6 = tf.get_variable('W_conv6', shape=[2, 2, n2, n2])
    b_conv6 = tf.get_variable('b_conv6', shape=[n2])
    h_conv6 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_2, W_conv6, strides=[1, 2, 2, 1], padding='SAME'), b_conv6))
    
    
    
    #block_3
    #dropout=0.2
    #cnn_7
    W_conv7 = tf.get_variable('W_conv7', shape=[2, 2, n2, n3])
    b_conv7 = tf.get_variable('b_conv7', shape=[n3])
    h_conv7 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv6, W_conv7, strides=[1, 1, 1, 1], padding='SAME'), b_conv7))
    #dropout_3
    d_3=tf.nn.dropout(h_conv6,0.8)
    #cnn_8
    W_conv8 = tf.get_variable('W_conv8', shape=[2, 2, n3, n3])
    b_conv8 = tf.get_variable('b_conv8', shape=[n3])
    h_conv8 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_3, W_conv8, strides=[1, 1, 1, 1], padding='SAME'), b_conv8))
    #dropout_4
    d_4=tf.nn.dropout(h_conv8,0.8)
    #cnn_9
    W_conv9 = tf.get_variable('W_conv9', shape=[2, 2, n3, n3])
    b_conv9 = tf.get_variable('b_conv9', shape=[n3])
    h_conv9 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_4, W_conv9, strides=[1, 2, 2, 1], padding='SAME'), b_conv9))
    

    
    #block_4
    #dropout=0.3
    #cnn_10
    W_conv10 = tf.get_variable('W_conv10', shape=[2, 2, n3, n4])
    b_conv10 = tf.get_variable('b_conv10', shape=[n4])
    h_conv10 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(h_conv9, W_conv10, strides=[1, 1, 1, 1], padding='SAME'), b_conv10))
    #dropout_5
    d_5=tf.nn.dropout(h_conv10,0.7)
    #cnn_11
    W_conv11 = tf.get_variable('W_conv11', shape=[2, 2, n4, n4])
    b_conv11 = tf.get_variable('b_conv11', shape=[n4])
    h_conv11 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_5, W_conv11, strides=[1, 1, 1, 1], padding='SAME'), b_conv11))
    #dropout_6
    d_6=tf.nn.dropout(h_conv11,0.7)
    #cnn_12
    W_conv12 = tf.get_variable('W_conv12', shape=[2, 2, n4, n4])
    b_conv12 = tf.get_variable('b_conv12', shape=[n4])
    h_conv12 = tf.nn.leaky_relu(tf.add(tf.nn.conv2d(d_5, W_conv12, strides=[1, 2, 2, 1], padding='SAME'), b_conv12))
    


    # fc layer to logits 10
    h_flat = tf.reshape(h_conv12, [-1, 8*8*n6])
    W_fc1 = tf.get_variable('W_fc1', shape=[8*8*n6, 10])
    b_fc1 = tf.get_variable('b_fc1', shape=[10])
    #layer of output
    logits = tf.add(tf.matmul(h_flat, W_fc1), b_fc1, name='logits')
    
    return(logits)

def compute_cross_entropy(logits, y):
    """compute the prediction and cross_entropy of model"""
    # This function is used from the in-class example code
    numerical_instability_example = 0
    if numerical_instability_example:
        y_pred = tf.nn.softmax(logits, name='y_pred') 
        cross_ent = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_pred), reduction_indices=[1]))
    else:
        sm_ce = tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits, name='cross_ent_terms')
        cross_ent = tf.reduce_mean(sm_ce, name='cross_ent')
    return cross_ent

def compute_accuracy(logits, y):
    "compare prediction to labels"
    prediction = tf.argmax(logits, 1, name='pred_class')
    true_label = tf.argmax(y, 1, name='true_class')
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, true_label), tf.float32))
    return accuracy
    
    

In [ ]:
dir_name = 'logs/scratch04x/summary'
with tf.Graph().as_default():

    x = tf.placeholder(tf.float32, [None, 32*32*3], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')

    
 
    with tf.name_scope('model'):
        logits = compute_logits(x)
    with tf.name_scope('loss'):
        loss = compute_cross_entropy(logits=logits, y=y)
    with tf.name_scope('accuracy'):
        accuracy = compute_accuracy(logits, y)
    
    with tf.name_scope('opt'):

        opt = tf.train.AdamOptimizer(1e-4)
        train_step = opt.minimize(loss)
    
    with tf.name_scope('summaries'):
        # create summary for loss and accuracy
        tf.summary.scalar('loss', loss) 
        tf.summary.scalar('accuracy', accuracy)
        # create summary for logits
        tf.summary.histogram('logits', logits)
        # create summary for input image
        tf.summary.image('input', tf.reshape(x, [-1, 32, 32, 3]))
    
        summary_op = tf.summary.merge_all()
    
    with tf.Session() as sess:
        summary_writer = tf.summary.FileWriter(dir_name, sess.graph)
        summary_writer_train = tf.summary.FileWriter(dir_name+'/train', sess.graph)
        summary_writer_test = tf.summary.FileWriter(dir_name+'/test')
        
        sess.run(tf.global_variables_initializer())
        batch=0
        for i in range(20001):
            
             
            _ , summary = sess.run((train_step, summary_op),
                                feed_dict={x: x_train[100*batch:100*(batch+1)], y: y_train0[100*batch:100*(batch+1)]})
            batch=batch+1
            if batch == 50:
                batch=0
            if i%10==0:
                t=rd.sample(range(0,10000),1000)
                summary_writer_train.add_summary(summary, i)
                (test_error, summary) = sess.run((accuracy,summary_op), {x:x_test[t], y:y_test0[t]})
                if i%1000 == 0:
                    print("\rAfter step {0:3d}, test accuracy {1:0.4f}".format(i, test_error), flush=True)

    

After step   0, test accuracy 0.0870
